In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
%matplotlib inline

In [45]:
all_fits_files = []
listOfFiles = os.listdir('./')
pattern = "*.fits"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_fits_files.append(entry)
            
all_hdr_files = []
listOfFiles = os.listdir('./')
pattern = "*.hdr"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_hdr_files.append(entry)


In [46]:
print(all_fits_files)
print(all_hdr_files)

[]
['fbs1477_cor.fits.hdr']


In [47]:
obj_df = pd.read_csv("astro_objects.csv");
obj_df.head()

,Unnamed: 0,Unnamed: 0.1,plate,object_id,ra,dec,mag_B,mag_R,spectrum_class,spectrum_length,...,9,10,11,12,13,14,15,16,fits_path,jpeg_path
0,0,0,fbs0005,DFBSJ142114.64+425823.6,14:21:14.640,+42:58:23.63,13.86,13.15,OK,101,...,-2.189981e-11,-3.665937e-14,-9.373043e-12,-1.532294e-09,1.024176e-11,-2.280213e-14,2.109259e-17,-6.999758e-21,specimages/fbs0005-DFBSJ142114.64+425823.6.fits,specimagesjpeg/fbs0005-DFBSJ142114.64+425823.6...
1,1,1,fbs0005,DFBSJ142122.01+425921.3,14:21:22.012,+42:59:21.37,12.85,12.20,OK,106,...,-3.792762e-11,-3.647929e-14,-1.208479e-11,-1.865856e-09,1.801383e-11,-4.706942e-14,4.814870e-17,-1.724178e-20,specimages/fbs0005-DFBSJ142122.01+425921.3.fits,specimagesjpeg/fbs0005-DFBSJ142122.01+425921.3...
2,2,2,fbs0005,DFBSJ142125.68+425138.2,14:21:25.688,+42:51:38.29,16.38,14.85,NL,83,...,-1.815388e-11,-1.309881e-14,-3.936086e-12,-6.570811e-10,3.287456e-12,-6.007407e-15,4.796814e-18,-1.419620e-21,specimages/fbs0005-DFBSJ142125.68+425138.2.fits,specimagesjpeg/fbs0005-DFBSJ142125.68+425138.2...
3,3,3,fbs0005,DFBSJ142126.80+425301.4,14:21:26.802,+42:53:01.40,14.45,13.57,NL,120,...,-4.077909e-11,-2.942382e-14,-8.841611e-12,-1.475995e-09,8.681218e-12,-1.793400e-14,1.573814e-17,-5.009265e-21,specimages/fbs0005-DFBSJ142126.80+425301.4.fits,specimagesjpeg/fbs0005-DFBSJ142126.80+425301.4...
4,4,4,fbs0005,DFBSJ142141.43+430014.5,14:21:41.436,+43:00:14.59,16.39,16.47,NL,90,...,-1.024508e-12,-1.714980e-15,-4.384846e-13,-7.168274e-11,5.490072e-13,-1.330955e-15,1.317058e-18,-4.632790e-22,specimages/fbs0005-DFBSJ142141.43+430014.5.fits,specimagesjpeg/fbs0005-DFBSJ142141.43+430014.5...


In [48]:
plate_id = all_hdr_files[0].split("_")[0]
print(plate_id)

fbs1477


In [49]:
this_plate_df = obj_df.loc[obj_df['plate'] == plate_id] 

In [50]:
this_plate_df.insert(0, "dx", 0)
this_plate_df.insert(0, "dy", 0)
this_plate_df.head()

,dy,dx,Unnamed: 0,Unnamed: 0.1,plate,object_id,ra,dec,mag_B,mag_R,...,9,10,11,12,13,14,15,16,fits_path,jpeg_path
746,0,0,746,746,fbs1477,DFBSJ081755.98+190314.9,08:17:55.984,+19:03:14.99,15.87,13.68,...,-4.146239e-11,-6.940651e-14,-1.774584e-11,-2.901079e-09,1.490176e-11,-2.834247e-14,2.360833e-17,-7.270340e-21,specimages/fbs1477-DFBSJ081755.98+190314.9.fits,specimagesjpeg/fbs1477-DFBSJ081755.98+190314.9...
747,0,0,747,747,fbs1477,DFBSJ081756.25+185537.2,08:17:56.255,+18:55:37.25,15.19,14.40,...,3.351549e-12,2.418313e-15,7.266861e-13,1.213128e-10,4.547560e-15,-8.282495e-16,1.165559e-18,-4.622449e-22,specimages/fbs1477-DFBSJ081756.25+185537.2.fits,specimagesjpeg/fbs1477-DFBSJ081756.25+185537.2...
748,0,0,748,748,fbs1477,DFBSJ081757.83+185724.2,08:17:57.830,+18:57:24.24,16.22,15.27,...,-6.516964e-12,-4.702264e-15,-1.412991e-12,-2.358810e-10,1.351952e-12,-2.763035e-15,2.417256e-18,-7.701931e-22,specimages/fbs1477-DFBSJ081757.83+185724.2.fits,specimagesjpeg/fbs1477-DFBSJ081757.83+185724.2...
749,0,0,749,749,fbs1477,DFBSJ081758.22+185818.0,08:17:58.226,+18:58:18.03,15.88,15.10,...,4.439892e-12,3.203587e-15,9.626533e-13,1.607039e-10,-4.890589e-13,5.019760e-16,-1.761059e-19,3.065624e-24,specimages/fbs1477-DFBSJ081758.22+185818.0.fits,specimagesjpeg/fbs1477-DFBSJ081758.22+185818.0...
750,0,0,750,750,fbs1477,DFBSJ081759.08+190328.7,08:17:59.083,+19:03:28.76,15.22,13.93,...,-2.153562e-11,-1.553885e-14,-4.669294e-12,-7.794794e-10,4.595515e-12,-9.488847e-15,8.317449e-18,-2.644268e-21,specimages/fbs1477-DFBSJ081759.08+190328.7.fits,specimagesjpeg/fbs1477-DFBSJ081759.08+190328.7...


In [59]:
import math
fits_files = WCS(all_hdr_files[0]) 
for index, row in this_plate_df.iterrows():
    #print(row['ra'], row['dec'])
    obj_ra = row['ra']#this_plate_df.loc[obj_loc_id]["ra"]
    obj_dec = row['dec']#this_plate_df.loc[obj_loc_id]["dec"] 
    c = SkyCoord(obj_ra, obj_dec, unit=(u.hourangle, u.deg))
    dx, dy = fits_files.all_world2pix(c.ra, c.dec, 0)
    print(dx, dy)
    this_plate_df.at[index, 'dx'] = math.floor(dx + 0.5)
    this_plate_df.at[index, 'dy'] = math.floor(dy + 0.5)
    #row["dx"] = dx
    #row["dy"] = dy

4928.6026005358735 4893.714762954662
4915.114227619987 4596.918935674307
4903.306594843678 4666.849849966508
4900.986532506978 4701.872969027294
4900.650669358823 4903.699079490029
4886.663170773239 4618.900352276106
4872.664712520675 4588.05135062585
4854.323326090123 4633.1439045749885
4852.488263977572 4870.64031391236
4845.537927111485 4778.381933055694
4814.809468910243 4876.675213050654
4804.020262220615 4705.972919065715
4797.809114290198 4794.361407509904
4772.688392723625 4623.414982684734
4756.129414352506 4588.450624155864
4741.904703142008 4647.591578457414
4731.531601870143 4867.653307073462
4685.6930648973785 4641.501947240581
4685.334065010088 4720.735572631072
4664.943985160534 4789.781234927378
4636.392336873954 4598.408110520916
4627.257336654471 4905.179519901166
4617.689384753662 4696.745378237693
4617.208528957824 4688.157695370737


In [61]:
#obj_loc_id = 746
#obj_ra = this_plate_df.loc[obj_loc_id]["ra"]
#obj_dec = this_plate_df.loc[obj_loc_id]["dec"] 
#c = SkyCoord(obj_ra, obj_dec, unit=(u.hourangle, u.deg))
#c
this_plate_df.head()

,dy,dx,Unnamed: 0,Unnamed: 0.1,plate,object_id,ra,dec,mag_B,mag_R,...,9,10,11,12,13,14,15,16,fits_path,jpeg_path
746,4894,4929,746,746,fbs1477,DFBSJ081755.98+190314.9,08:17:55.984,+19:03:14.99,15.87,13.68,...,-4.146239e-11,-6.940651e-14,-1.774584e-11,-2.901079e-09,1.490176e-11,-2.834247e-14,2.360833e-17,-7.270340e-21,specimages/fbs1477-DFBSJ081755.98+190314.9.fits,specimagesjpeg/fbs1477-DFBSJ081755.98+190314.9...
747,4597,4915,747,747,fbs1477,DFBSJ081756.25+185537.2,08:17:56.255,+18:55:37.25,15.19,14.40,...,3.351549e-12,2.418313e-15,7.266861e-13,1.213128e-10,4.547560e-15,-8.282495e-16,1.165559e-18,-4.622449e-22,specimages/fbs1477-DFBSJ081756.25+185537.2.fits,specimagesjpeg/fbs1477-DFBSJ081756.25+185537.2...
748,4667,4903,748,748,fbs1477,DFBSJ081757.83+185724.2,08:17:57.830,+18:57:24.24,16.22,15.27,...,-6.516964e-12,-4.702264e-15,-1.412991e-12,-2.358810e-10,1.351952e-12,-2.763035e-15,2.417256e-18,-7.701931e-22,specimages/fbs1477-DFBSJ081757.83+185724.2.fits,specimagesjpeg/fbs1477-DFBSJ081757.83+185724.2...
749,4702,4901,749,749,fbs1477,DFBSJ081758.22+185818.0,08:17:58.226,+18:58:18.03,15.88,15.10,...,4.439892e-12,3.203587e-15,9.626533e-13,1.607039e-10,-4.890589e-13,5.019760e-16,-1.761059e-19,3.065624e-24,specimages/fbs1477-DFBSJ081758.22+185818.0.fits,specimagesjpeg/fbs1477-DFBSJ081758.22+185818.0...
750,4904,4901,750,750,fbs1477,DFBSJ081759.08+190328.7,08:17:59.083,+19:03:28.76,15.22,13.93,...,-2.153562e-11,-1.553885e-14,-4.669294e-12,-7.794794e-10,4.595515e-12,-9.488847e-15,8.317449e-18,-2.644268e-21,specimages/fbs1477-DFBSJ081759.08+190328.7.fits,specimagesjpeg/fbs1477-DFBSJ081759.08+190328.7...


In [53]:
#dx, dy = fits_files.all_world2pix(c.ra, c.dec, 0)

In [54]:
#print(dx)
#print(dy)

In [62]:
this_plate_df.to_csv("plate_fbs1477_objects.csv")